In [1]:
from typing import Annotated
import operator,json
from typing import TypedDict, Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph,END,START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [3]:
llm.invoke("hi")

AIMessage(content='Hi there! 👋  How can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 10, 'total_tokens': 25, 'completion_time': 0.027272727, 'prompt_time': 4.1e-07, 'queue_time': 0.023117989000000002, 'total_time': 0.027273137}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b2c10669-d41a-4fa1-90c5-49fafd86ee4d-0', usage_metadata={'input_tokens': 10, 'output_tokens': 15, 'total_tokens': 25})

In [4]:
llm.invoke("hi").content

'Hello! 👋\n\nHow can I help you today?\n'

In [5]:
@tool
def multiply(first_number:int, second_number:int)->int:
    """multiply two integer number"""
    return first_number * second_number

In [6]:
multiply({"first_number":20,"second_number":27})

C:\Users\Toshiba\AppData\Local\Temp\ipykernel_13224\623416424.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  multiply({"first_number":20,"second_number":27})


540

In [7]:
multiply.invoke({"first_number":20,"second_number":27})

540

In [8]:
@tool
def search(query:str):
    """perform the web search on the user query"""
    tavily=TavilySearchResults()
    result=tavily.invoke(query)
    return result

In [9]:
search("who is the new president of USA?")

[{'url': 'https://www.bbc.co.uk/newsround/articles/ce3nq0rexqeo',
  'content': 'Donald Trump officially becomes new US president - BBC Newsround Donald Trump officially becomes new US president In a special ceremony called an inauguration, President Trump took over from Joe Biden as America\'s new leader. In his first speech as president after swearing the oath, Mr Trump promised to make America "greater, stronger, and far more exceptional than ever before". It\'s the second time that Mr Trump has taken on the role - having previously served as president before Joe Biden, between 2017-2021. Donald Trump: What happens at the inauguration of the new US president? What are Donald Trump\'s plans as president? Following his oath, Mr Trump gave his first speech as president - which is known as the Inaugural Address.'},
 {'url': 'https://www.usnews.com/news/politics/articles/2025-01-20/inauguration-day-latest-biden-pardons-fauci-milley-and-jan-6-committee-before-trump-takes-office',
  'conten

In [10]:
search.invoke("who is the new president of USA?")

[{'url': 'https://www.bbc.co.uk/newsround/articles/ce3nq0rexqeo',
  'content': 'Donald Trump officially becomes new US president - BBC Newsround Donald Trump officially becomes new US president In a special ceremony called an inauguration, President Trump took over from Joe Biden as America\'s new leader. In his first speech as president after swearing the oath, Mr Trump promised to make America "greater, stronger, and far more exceptional than ever before". It\'s the second time that Mr Trump has taken on the role - having previously served as president before Joe Biden, between 2017-2021. Donald Trump: What happens at the inauguration of the new US president? What are Donald Trump\'s plans as president? Following his oath, Mr Trump gave his first speech as president - which is known as the Inaugural Address.'},
 {'url': 'https://www.usa.gov/presidents',
  'content': 'Vice president of the United States\nThe vice president of the United States presides over the U.S. Senate and takes o

In [11]:
tools=[search,multiply]

In [12]:
model_with_tools = llm.bind_tools(tools)

In [13]:
tool_mapping={tool.name: tool for tool in tools}

In [14]:
tool_mapping

{'search': StructuredTool(name='search', description='perform the web search on the user query', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x000001BE28D75090>),
 'multiply': StructuredTool(name='multiply', description='multiply two integer number', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000001BE28D74B80>)}